In [1]:
import pandas as pd
import time
from Functions import xbrl_dict,filings_collector
import datetime as dt
import requests
import xml.etree.ElementTree as ET
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_

In [ ]:
ticker = input('ticker?').lower()

In [2]:
# Access page
all_filings_df = filings_collector(ticker)
all_filings_df.to_csv(f'../data/all_{ticker}_filings.csv', index=False)
all_filings_path = f'../data/all_{ticker}_filings.csv'

ticker?ba


In [7]:
all_filings_path = f'../data/all_{ticker}_filings.csv'
xml_filings_df = pd.read_csv(all_filings_path)
xml_filings_df = xml_filings_df.dropna(subset=['xbrl_link'])
xml_filings_df = xml_filings_df.reset_index(drop=True)
xml_filings_df.head(100)

,date,doc_link,filings,xbrl_link
0,2019-02-08,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
1,2018-10-24,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
2,2018-07-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
3,2018-04-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
4,2018-02-12,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
5,2017-10-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
6,2017-07-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
7,2017-04-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
8,2017-02-08,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
9,2016-10-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...


In [8]:
# lxml_ten_df = xml_filings_df.loc[xml_filings_df['filings'] == '10-Q']
filings_list = []
# for i in range(0,len(xml_filings_df)):
for i in range(0,12):
    lxml_link = xml_filings_df['xbrl_link'][i]

    lxml_dict = xbrl_dict(lxml_link)
    
    filings_list.append(lxml_dict)


In [12]:
all_data_df = pd.DataFrame(filings_list)
columns = all_data_df.columns.tolist()

In [13]:
establising_headers = ['company_name', 'CIK','doc_date',
              'doc_year','doc_period','common_shares_outstanding',
              'amendment_flag']
for header in establising_headers:
    index = columns.index(header)
    columns.pop(index)

ordered_columns = establising_headers+columns


In [14]:
all_data_df = all_data_df[ordered_columns]
all_data_df.tail(20)

,company_name,CIK,doc_date,doc_year,doc_period,common_shares_outstanding,amendment_flag,AccrualForEnvironmentalLossContingenciesIncreaseDecreaseForRevisionInEstimates,AccrualForEnvironmentalLossContingenciesPayments,AccrualForEnvironmentalLossContingenciesPayments1,...,UnrecognizedTaxBenefitsDecreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfShareOutstandingBasicAndDiluted,WeightedAverageNumberOfSharesOutstandingBasic,exceptions
0,BOEING CO,0000012927,2018-12-31,2018,FY,564985109,false,-68000000.0,NaN,37000000.0,...,410000000.0,206000000.0,1.208000e+09,87000000.0,3000000.0,6300000.0,585500000,579900000,579200000,106
1,BOEING CO,0000012927,2018-09-30,2018,Q3,567885369,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,580200000,574800000,574200000,3
2,BOEING CO,0000012927,2018-06-30,2018,Q2,574508314,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6100000.0,588000000,582600000,581900000,2
3,BOEING CO,0000012927,2018-03-31,2018,Q1,582580973,false,-22000000.0,NaN,7000000.0,...,NaN,NaN,NaN,NaN,NaN,6400000.0,596500000,590800000,590100000,50
4,BOEING CO,0000012927,2017-12-31,2017,FY,588490313,false,-7000000.0,NaN,45000000.0,...,44000000.0,NaN,2.200000e+08,3000000.0,NaN,7500000.0,610000000,603200000,602500000,106
5,BOEING CO,0000012927,2017-09-30,2017,Q3,595578523,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8000000.0,605600000,598300000,597600000,7
6,BOEING CO,0000012927,2017-06-30,2017,Q2,591083386,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7100000.0,608900000,602500000,601800000,4
7,BOEING CO,0000012927,2017-03-31,2017,Q1,603580992,false,26000000.0,NaN,11000000.0,...,NaN,NaN,NaN,NaN,NaN,6800000.0,620400000,614400000,613600000,40
8,BOEING CO,0000012927,2016-12-31,2016,FY,612478643,false,-43000000.0,NaN,47000000.0,...,348000000.0,73000000.0,3.440000e+08,17000000.0,NaN,7300000.0,642800000,636500000,635500000,103
9,BOEING CO,0000012927,2016-09-30,2016,Q3,617165902,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7200000.0,631800000,625500000,624600000,2


In [ ]:
all_years = []
for year in list(all_data_df['doc_year']):
    if year not in all_years:
         all_years.append(year)
counter = 1            
for index, row in all_data_df.iterrows():
#     year = row['doc_year']
#     period = row['doc_period']
    counter += 1
    print(row)
    print(counter)
    
    
    

In [15]:
all_data_df.to_csv(f'../data/{ticker}_financials.csv', index=False)

In [ ]:
# lxml_link = xml_filings_df['xbrl_link'][10]
print(lxml_link)

In [ ]:
# uris = []
# all_names = []
# for i in range(0,len(xml_filings_df)):
#     lxml_link = xml_filings_df['xbrl_link'][i]
#     xbrl_resp = requests.get(lxml_link)
#     time.sleep(.1)
#     xbrl_str = xbrl_resp.text
#     root = ET.fromstring(xbrl_str)

#     #     defining namespace prefixes
#     uri_list = []
#     names = []
#     for child in root:
#         uri_pos = child.tag.index('}')
#         uri = ''.join(child.tag[1:uri_pos])
#         name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])
        
#         if uri not in uri_list:
#             uri_list.append(uri)
        
#         if uri[0:18] == 'http://xbrl.us/dei':
#             names.append(f'{name}: {child.text}')
#     uris.append(uri_list)
#     all_names.append(names)

In [ ]:
lxml_link = xml_filings_df['xbrl_link'][0]
xbrl_resp = requests.get(lxml_link)
xbrl_str = xbrl_resp.text
root = ET.fromstring(xbrl_str)
uri_list = []
names = []
sec_list = []
for child in root:
    uri_pos = child.tag.index('}')
    uri = ''.join(child.tag[1:uri_pos])
    name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])

    if uri not in uri_list:
        uri_list.append(uri)

    if uri[0:23] == 'http://fasb.org/us-gaap':
        ref = child.attrib['contextRef']
        keys_test = {'ref' : ref, 'name' : name, 'data' : child.text}
        names.append(keys_test)  
    if uri[0:23] == 'http://xbrl.sec.gov/dei':
        sec_keys = {'name' : name, 'attribs' : child.attrib, 'data' : child.text}
        sec_list.append(sec_keys)

In [ ]:
sec_list

In [ ]:
# names
q1_names_df = pd.DataFrame(names)
q1_names_df.head()

In [ ]:
q1_names_df.to_csv(f'../data/{ticker}_xml_test.csv', index=False)

In [ ]:
engine = create_engine(root@127.0.0.1:3306